In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from __future__ import print_function
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from datetime import datetime
import numpy as np
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy import stats




/Users/abhishekkuntal/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import pandas as pd
data_bots = pd.read_csv('bots.csv')
data_nonbots = pd.read_csv('nonbots.csv')
frames = [data_bots, data_nonbots]
result = pd.concat(frames)


In [5]:
result.bot.value_counts()

0    1176
1    1056
Name: bot, dtype: int64

In [6]:
result.shape

(2232, 20)

In [3]:
mod_df = result.copy()
scratch = mod_df.copy()
scratch['age'] = 0;
#Check for not null values 
a = scratch['created_at']
pd.isnull(a).values.any()

False

In [4]:
#cleaning the dataframe containing dates based on the data we saw 
scratch['created_at'] = scratch['created_at'].map(lambda x: x.lstrip('"').rstrip('"'))

In [5]:
#converting to datetime format of pandas for age calculation 
scratch['created_at'] = pd.to_datetime(scratch['created_at'], coerce=True)

/Users/abhishekkuntal/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: the coerce=True keyword is deprecated, use errors='coerce' instead
  from ipykernel import kernelapp as app


In [6]:
c_date = scratch['created_at']
#Check the conversion went through smoothly
c_date.shape

(2232,)

In [7]:
#Check for non null values
nulls=(c_date.notnull()==False)
nulls.value_counts()

False    2232
Name: created_at, dtype: int64

In [8]:
x = scratch[['age']].copy()
now = datetime.now()
now = now.date()
x['age'] = now - c_date


In [9]:
#Converting date time to numeric for calculation 
x['age'] = x['age'].astype('timedelta64[D]')

In [10]:
#find outliers 
#X = x['age'].astype('timedelta64[D]')
X = x['age'].copy()
X['C_sign'] = np.sign(X)
###Value counts to see any outliers for the data
X['C_sign'].value_counts()

 1.0    2213
-1.0      19
Name: age, dtype: int64

In [11]:
indx = X['C_sign'] > 0
indx.value_counts()

True     2213
False      19
Name: age, dtype: int64

In [15]:
### Deal with the outliers of the data convert it to the statistical mean of the data check how to do that using other attributes 
### for the time being fill it with mean 

z = indx[indx == False].index.tolist()
scratch['created_at'].iloc[z]

15    2017-12-03 14:59:00
41    2017-12-03 18:47:00
127   2017-12-03 15:18:00
194   2017-12-03 19:19:00
236   2017-12-03 04:12:00
302   2017-12-03 18:26:00
345   2017-12-03 18:48:00
364   2017-09-03 04:08:00
401   2017-09-03 16:25:00
498   2017-11-03 07:07:00
513   2017-12-03 19:23:00
549   2017-09-03 16:01:00
653   2017-10-03 13:46:00
727   2017-12-03 19:06:00
735   2017-11-03 10:48:00
762   2017-12-03 11:58:00
764   2017-09-03 02:42:00
898   2017-12-03 04:27:00
950   2017-12-03 18:55:00
Name: created_at, dtype: datetime64[ns]

In [16]:
#see the values at those index
# outliers values replace it with mean should be fine for now
x.iloc[z]

,age
15,-239.0
41,-239.0
127,-239.0
194,-239.0
236,-239.0
302,-239.0
345,-239.0
364,-148.0
401,-148.0
498,-209.0


In [52]:
#code to find not null indexes
#inds = pd.isnull(y).nonzero()[0]
#code for seeing the data in those rows and coloumns
#scratch['created_at'].iloc[inds]
#ind = pd.isnull(indx).nonzero()[0]

In [17]:
scratch['age'] = x['age']

In [18]:
checking = x[['age']].copy()


In [19]:
#replace outlier with nan for correct mean calculation
checking.iloc[z,0] = np.nan
#replace with mean for now. Todo find a better way to replace them. 
checking['age'] = checking['age'].fillna(checking['age'].mean())

In [20]:
#check values substituted
checking['age'].iloc[z]

15     1607.181202
41     1607.181202
127    1607.181202
194    1607.181202
236    1607.181202
302    1607.181202
345    1607.181202
364    1607.181202
401    1607.181202
498    1607.181202
513    1607.181202
549    1607.181202
653    1607.181202
727    1607.181202
735    1607.181202
762    1607.181202
764    1607.181202
898    1607.181202
950    1607.181202
Name: age, dtype: float64

In [21]:
#Check if the values went through properly 
# del X
# X = checking['age'].copy()
# X['C_sign'] = np.sign(X)
# ###Value counts to see any outliers for the data
# X['C_sign'].value_counts()

In [22]:
scratch['age'] = checking['age']

In [23]:
#Shuffle the dataset for using  the dataset 
#shuffled = scratch
#shuffled = shuffled.sample(frac=1).reset_index(drop=True)

In [24]:
# X = shuffled['age'].copy()
# X['C_sign'] = np.sign(X)
# ###Value counts to see any outliers for the data
# X['C_sign'].value_counts()

In [25]:
# indx = X['C_sign'] > 0
# z = indx[indx == False].index.tolist()
# shuffled['created_at'].iloc[z]

In [26]:
r_forest = scratch.copy()

In [27]:
#(r_forest.columns[['id','id_str','screen_name','location','description','url','name']],
#r_forest.fillna(r_forest.mean())
r_forest['has_extended_profile'] = r_forest['has_extended_profile'].fillna(0)

In [28]:
r_forest.drop('id', axis = 1, inplace = True)
r_forest.drop('id_str', axis = 1, inplace = True)
r_forest.drop('screen_name', axis = 1, inplace = True)
r_forest.drop('location', axis = 1, inplace = True)
r_forest.drop('description', axis = 1, inplace = True)
r_forest.drop('url', axis = 1, inplace = True)
r_forest.drop('name', axis = 1, inplace = True)
r_forest.drop('status', axis = 1, inplace = True)
r_forest.drop('created_at', axis = 1, inplace = True)
r_forest.drop('lang', axis =1, inplace = True)

In [29]:
r_forest = r_forest.sample(frac=1).reset_index(drop=True)

In [30]:
r_forest.shape

(2232, 11)

In [31]:
r_forest.verified = r_forest.verified.astype(int)
r_forest.default_profile = r_forest.default_profile.astype(int)
r_forest.default_profile_image = r_forest.default_profile_image.astype(int)

In [32]:
#swap age and bots 
c = list(r_forest)
c[9], c[10] = c[10], c[9]
r_forest.columns = c
r_forest[[c[9],c[10]]] = r_forest[[c[10],c[9]]]

In [33]:
r_forest.head(10)

,followers_count,friends_count,listedcount,favourites_count,verified,statuses_count,default_profile,default_profile_image,has_extended_profile,age,bot
0,291,1736,7,0,0,543,1,1,False,2193.0,0
1,151,1833,4,164,0,591,0,0,False,1075.0,0
2,1059,3599,254,1150,0,17857,0,0,True,2920.0,0
3,1132,8,7,32,0,656,1,0,True,143.0,0
4,9,25,0,400,0,385,1,0,False,62.0,1
5,879,1,151,2426,0,1217604,1,0,False,1794.0,1
6,10943449,501,23938,186,1,547,0,0,False,2417.0,0
7,10,1,5,0,0,13148,1,0,False,767.0,1
8,29879,108,88,1589,0,50659,0,0,False,1673.0,1
9,349930,845,4885,70,1,20775,1,0,False,2935.0,0


In [34]:
#check for any null values in the trimmed dataset 
pd.isnull(r_forest).any(1).nonzero()[0]

array([], dtype=int64)

In [35]:
#split the dataset 
r_train, r_test, rl_train, rl_test = train_test_split(r_forest.ix[:,0:10], r_forest['bot'], test_size=0.2, random_state = 2)

In [36]:
r_train.head(1)

,followers_count,friends_count,listedcount,favourites_count,verified,statuses_count,default_profile,default_profile_image,has_extended_profile,age
1728,2,362,0,34,0,39,1,0,False,131.0


In [37]:
rf = RandomForestClassifier(n_estimators=30)
rf.fit(r_train, rl_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=30, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [38]:
predicted = rf.predict(r_test)

In [47]:
print("Precision Score =", metrics.precision_score(predicted, rl_test))
print("Accuracy Score  =", metrics.accuracy_score(predicted, rl_test))
print("Recall Score    =", metrics.recall_score(predicted, rl_test))
print("F1 Score        =", metrics.f1_score(predicted, rl_test))
print("AUC Score       =", metrics.roc_auc_score(predicted, rl_test))

Precision Score = 0.926605504587
Accuracy Score  = 0.939597315436
Recall Score    = 0.948356807512
F1 Score        = 0.937354988399
AUC Score       = 0.939990369568


In [41]:
r_train['has_extended_profile'].value_counts()

False    1528
True      257
Name: has_extended_profile, dtype: int64

In [42]:
pca_a = r_forest.copy()

pca_X = pca_a.ix[:,0:10]
pca_X.head(2)

,followers_count,friends_count,listedcount,favourites_count,verified,statuses_count,default_profile,default_profile_image,has_extended_profile,age
0,291,1736,7,0,0,543,1,1,False,2193.0
1,151,1833,4,164,0,591,0,0,False,1075.0


In [43]:
# using pca to find which attributes to drop 
pca_std = StandardScaler().fit_transform(pca_X)
sklearn_pca = sklearnPCA(0.90)
Y_sklearn = sklearn_pca.fit_transform(pca_std)
Y_sklearn.shape

(2232, 8)